In [ ]:
%matplotlib inline
import carmcmc as cm
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic
import glob, os
import time
from matplotlib.backends.backend_pdf import PdfPages
    
start = time.clock()    
infor=np.genfromtxt('/Users/legolason/study/lightcurve/SF_test/DB_QSO_S82.dat')
ind_BHnozero=list(np.array(np.where(infor[:,7]) ).reshape(-1,))
ind_Minozero=list(np.array(np.where((infor[:,5]) < -5 )).reshape(-1,))
index=list(set(ind_BHnozero).intersection(ind_Minozero))

ID=infor[index,0]
M_i=infor[index,5]
M_bh=infor[index,7]
z=infor[index,6]
wave_rf=7625./(1+z)

infor_id=infor[index,0]
infor_ra=infor[index,1]
infor_dec=infor[index,2]
infor_z=infor[index,6]
infor_EBV=infor[index,14]
#print infor_id


#os.chdir("/Users/legolason/study/lightcurve/SF_test/data")
os.chdir("/Users/legolason/study/lightcurve/stripe82/QSO_S82")
all_file=np.sort(map(int,np.asarray(glob.glob("*"))))
#print all_file
c=0
mu=[]
log_omega=[]
sigma=[]
measerr_scale=[]
var=[]
tau=[]

for file in all_file:
    data=np.genfromtxt(str(all_file[c]))
    count_lines = len(open(str(file)).readlines())  
    #print count_lines
    if count_lines < 20:             #delete the epoch less than a certain number
        c+=1
        continue
    #print all_file[c],file,'----------'
    ra=data[0,15]
    dec=data[0,16]
    
    #check whether they are the same source
    #index_list=np.where(ra-infor_ra[c] < 2./3600. and dec-infor_dec[c] < 2./3600.)
    #print ra-infor_ra[c],dec-infor_dec[c]

    juldata=data[:,0]
    t=juldata-juldata.min()

    #---------u band------------
    mag_u=data[:,1]
    index_u=np.where((mag_u>0) & (mag_u <30))
    mag_u=mag_u[index_u]+infor_EBV[c]   #Ag,Ar,Ai,Az=0.736,0.534,0.405,0.287*Au   correct galactic extinction--
    mag_u_err=data[:,2]
    mag_u_err=mag_u_err[index_u]
    t_u=t[index_u]
    
    model = cm.CarmaModel(t_u, mag_u, mag_u_err, p=1, q=0)
    sample = model.run_mcmc(20000)
    #sample.assess_fit() 
    #print sample.parameters
    mu.append(sample.get_samples('mu').mean())
    log_omega.append(sample.get_samples('log_omega').mean())
    sigma.append(sample.get_samples('sigma').mean())
    var.append(sample.get_samples('var').mean())
    measerr_scale.append(sample.get_samples('measerr_scale').mean())
    tau.append((sample.get_samples('var')*2/sample.get_samples('sigma')**2).mean())
    print tau,sigma


    
    #---------g band------------
    mag_g=data[:,4]
    index_g=np.where(mag_g>0)
    mag_g=mag_g[index_g]+infor_EBV[c]*0.736
    mag_g_err=data[:,5]
    mag_g_err=mag_g_err[index_g]
    t_g=t[index_g]
  

    #---------r band------------
    mag_r=data[:,7]
    index_r=np.where(mag_r>0)
    mag_r=mag_r[index_r]+infor_EBV[c]*0.534
    mag_r_err=data[:,8]
    mag_r_err=mag_r_err[index_r]
    t_r=t[index_r]

    #---------i band------------
    mag_i=data[:,10]
    index_i=np.where(mag_i>0)
    mag_i=mag_i[index_i]+infor_EBV[c]*0.405
    mag_i_err=data[:,11]
    mag_i_err=mag_i_err[index_i]
    t_i=t[index_i]
    

    #---------z band------------
    mag_z=data[:,13]
    index_z=np.where(mag_z>0)
    mag_z=mag_z[index_z]+infor_EBV[c]*0.287
    mag_z_err=data[:,14]
    mag_z_err=mag_z_err[index_z]
    t_z=t[index_z]

    # plot light curve for ugriz bands
    fig1=plt.figure(1)
    plt.rc('font', size=6)
    plt.subplot(2,2,1)
    plt.errorbar(t_u,mag_u,yerr=mag_u_err,color='b')
    plt.plot(t_u, mag_u, 'b.',label="u")
    plt.errorbar(t_g,mag_g,yerr=mag_g_err,color='g')
    plt.plot(t_g, mag_g, 'g.',label="g")
    plt.errorbar(t_r,mag_r,yerr=mag_r_err,color='r')
    plt.plot(t_r, mag_r, 'r.',label="r")
    plt.errorbar(t_i,mag_i,yerr=mag_i_err,color='y')
    plt.plot(t_i, mag_i, 'y.',label="i")
    plt.errorbar(t_z,mag_z,yerr=mag_z_err,color='k')
    plt.plot(t_z, mag_z, 'k.',label="z")
    plt.xlabel('T (days)')
    plt.ylabel('Magnitude')
    plt.title('ID:'+str(all_file[c]),color='k')
    #plt.legend()


    n=len(t_u)
    tau_u = [t_u[i]-t_u[j] for i in range(n) for j in range(i)]
    tau_u = np.array(tau_u)
    delta_mag_u = [np.abs(mag_u[i]-mag_u[j]) for i in range(n) for j in range(i)]
    delta_mag_u = np.array(delta_mag_u)


    n=len(t_g)
    tau_g = [t_g[i]-t_g[j] for i in range(n) for j in range(i)]
    tau_g = np.array(tau_g)
    delta_mag_g = [np.abs(mag_g[i]-mag_g[j]) for i in range(n) for j in range(i)]
    delta_mag_g = np.array(delta_mag_g)

    n=len(t_r)
    tau_r = [t_r[i]-t_r[j] for i in range(n) for j in range(i)]
    tau_r = np.array(tau_r)
    delta_mag_r = [np.abs(mag_r[i]-mag_r[j]) for i in range(n) for j in range(i)]
    delta_mag_r = np.array(delta_mag_r)

    n=len(t_i)
    tau_i = [t_i[i]-t_i[j] for i in range(n) for j in range(i)]
    tau_i = np.array(tau_i)
    delta_mag_i = [np.abs(mag_i[i]-mag_i[j]) for i in range(n) for j in range(i)]
    delta_mag_i = np.array(delta_mag_i)


    n=len(t_z)
    tau_z = [t_z[i]-t_z[j] for i in range(n) for j in range(i)]
    tau_z = np.array(tau_z)
    delta_mag_z = [np.abs(mag_z[i]-mag_z[j]) for i in range(n) for j in range(i)]
    delta_mag_z = np.array(delta_mag_z)

    plt.subplot(2,2,2)
    plt.plot(tau_u,delta_mag_u,'b.')
    plt.plot(tau_g,delta_mag_g,'g.')
    plt.plot(tau_r,delta_mag_r,'r.')
    plt.plot(tau_i,delta_mag_i,'y.')
    plt.plot(tau_z,delta_mag_z,'k.')
    plt.xlabel('$\Delta$ t (days)')
    plt.ylabel('$\Delta$ Mag')



    def func(x):return np.sqrt((x**2).mean())
    mag_rms_u,bin_edges_u,num_u = binned_statistic(tau_u, delta_mag_u, bins=10,statistic=func,range=(0, 4000))
    x_u = (bin_edges_u[:-1]+bin_edges_u[1:])/2
    mag_rms_g,bin_edges_g,num_g = binned_statistic(tau_g, delta_mag_g, bins=10,statistic=func,range=(0, 4000))
    x_g = (bin_edges_g[:-1]+bin_edges_g[1:])/2
    mag_rms_r,bin_edges_r,num_r = binned_statistic(tau_r, delta_mag_r, bins=10,statistic=func,range=(0, 4000))
    x_r = (bin_edges_r[:-1]+bin_edges_r[1:])/2
    mag_rms_i,bin_edges_i,num_i = binned_statistic(tau_i, delta_mag_i, bins=10,statistic=func,range=(0, 4000))
    x_i = (bin_edges_i[:-1]+bin_edges_i[1:])/2
    mag_rms_z,bin_edges_z,num_z = binned_statistic(tau_z, delta_mag_z, bins=10,statistic=func,range=(0, 4000))
    x_z = (bin_edges_z[:-1]+bin_edges_z[1:])/2


    plt.subplot(2,2,3)
    plt.plot(x_u,mag_rms_u,'bo-')
    plt.plot(x_g,mag_rms_g,'go-')
    plt.plot(x_r,mag_rms_r,'ro-')
    plt.plot(x_i,mag_rms_i,'yo-')
    plt.plot(x_z,mag_rms_z,'ko-')
    plt.xlabel('$\Delta$ t (days)')
    plt.ylabel('$\Delta$ Mag')    
    plt.tight_layout()
    
    
    fig2=plt.figure(2)
    sample.assess_fit()
    
    #save data
    pp = PdfPages('/Users/legolason/study/lightcurve/SF_test/lc_eps/lc_'+str(all_file[c])+'.pdf')
    pp.savefig(fig1)
    plt.close()
    pp.savefig(fig2)
    plt.close()
    pp.close()
    end = time.clock()
    print c,all_file[c]
    c+=1    
np.savetxt('/Users/legolason/study/lightcurve/SF_test/para/para.txt', (tau,var,mu,sigma,log_omega,measerr_scale))
print "timeused:"+str(end-start)+'s'    

In [ ]:
#fit the observed lightcurve with car1 get the tau sigma and their error, save them in many files.
%matplotlib inline
import carmcmc as cm
import numpy as np
import matplotlib.pyplot as plt
import glob, os

       
infor=np.genfromtxt('/Users/legolason/study/lightcurve/SF_test/DB_QSO_S82.dat')

#normal BH, Mi select index
ind_BHnozero=list(np.array(np.where(infor[:,7]) ).reshape(-1,))
ind_Minozero=list(np.array(np.where((infor[:,5]) < -5 )).reshape(-1,))
index=list(set(ind_BHnozero).intersection(ind_Minozero))

ID=infor[index,0]
ra=infor[index,1]
dec=infor[index,2]
z=infor[index,6]
EBV=infor[index,14]


#os.chdir("/Users/legolason/study/lightcurve/SF_test/data")
os.chdir("/Users/legolason/study/lightcurve/stripe82/QSO_S82")
all_file=np.sort(map(int,np.asarray(glob.glob("*"))))


c=0
 
for i in range(len(all_file[index])):
    data=np.genfromtxt(str(all_file[i]))

    count_lines = len(open(str(all_file[i])).readlines())  
    if count_lines < 20:  #delete the epoch less than a certain number
        c+=1
        continue
    mag_i=data[:,10]
    index_i=np.where(mag_i>0)
    mag_i=mag_i[index_i]
    mag_i_err=data[:,11]
    mag_i_err=mag_i_err[index_i]
    
    
    juldata=data[:,0]
    t=juldata-juldata.min()
    t_i=t[index_i]
    model = cm.CarmaModel(t_i, mag_i, mag_i_err, p=1, q=0)
    
    sample = model.run_mcmc(20000)
    tau=np.exp(-sample.get_samples('log_omega'))/(1+z[i])
    sigma=sample.get_samples('sigma')
    mu=sample.get_samples('mu')
    var=sample.get_samples('var')
    measerr_scale=sample.get_samples('measerr_scale')
    print i,all_file[i],tau.mean()
    np.savetxt('/Users/legolason/study/lightcurve/SF_test/para/car1/'+str(all_file[i])+'.txt',\
               np.c_[tau,sigma,var,mu,measerr_scale],fmt='%8.4e',header='tau_rest, sigma, var,mu,measerr_scale')


In [ ]:
#read files of car1 results from 5750 qsos (126 less than 20 epochs) in to one file
%matplotlib inline
import carmcmc as cm
import numpy as np
import matplotlib.pyplot as plt
import glob, os

os.chdir("/Users/legolason/study/lightcurve/SF_test/para/car1")
all_file=np.sort(map(int,np.asarray(glob.glob("*"))))


tau=np.zeros(len(all_file))
tau_low=np.zeros(len(all_file))
tau_up=np.zeros(len(all_file))
sigma=np.zeros(len(all_file))
sig_low=np.zeros(len(all_file))
sig_up=np.zeros(len(all_file))
SF=np.zeros(len(all_file))
var=np.zeros(len(all_file))
mu=np.zeros(len(all_file))
measerr_scale=np.zeros(len(all_file))



for i in range(len(all_file)):
    data=np.genfromtxt(str(all_file[i]))
    print i
    tau[i]=int(np.median(data[:,0]))
    tau_low[i]=int(np.percentile(data[:,0],(16,84))[0])
    tau_up[i]=int(np.percentile(data[:,0],(16,84))[1])
    sigma[i]=round(np.median(data[:,1]),3)
    sig_low[i]=round(np.percentile(data[:,1],(16,84))[0],3)
    sig_up[i]=round(np.percentile(data[:,1],(16,84))[1],3)
    SF[i]=np.median(data[:,1])*np.sqrt(np.median(data[:,0]))
    var[i]=np.median(data[:,2])
    mu[i]=np.median(data[:,3])
    measerr_scale[i]=np.median(data[:,4])
    
np.savez('/Users/legolason/study/lightcurve/SF_test/para/car1_para_obs.txt',\
           np.c_[tau,tau_low,tau_up,sigma,sig_low,sig_up,SF,var,mu,measerr_scale],\
          header='rest_tau,tau_low,tau_up,sigma,sig_low,sig_up,SF,var,mu,measerr_scale',fmt='%4.3f')
#np.savetxt('/Users/legolason/study/lightcurve/SF_test/para/car1_para_obs.txt',\
#           np.c_[tau,tau_low,tau_up,sigma,sig_low,sig_up,SF,var,mu,measerr_scale],\
#          header='rest_tau,tau_low,tau_up,sigma,sig_low,sig_up,SF,var,mu,measerr_scale',fmt='%4.3f')
print 'successed!'
    

In [ ]:
# test lightcurve compare two methods!  1st!
%matplotlib inline
import carmcmc as cm
import numpy as np
import matplotlib.pyplot as plt
import os,glob
from javelin.zylc import get_data
from javelin.lcmodel import Cont_Model

def model_lc(time, mu, tau, sigma):
    '''re-generate a new light cuvre with given tau, sigma and mu '''
    t=time
    E_mag=np.zeros(len(t))
    E_mag[0]=mu
    var=np.zeros(len(t))
    var[0]=tau*sigma**2/2.
    X=np.zeros(len(t))
    X[0]=mu
    dt=np.zeros(len(t))
    X_err=np.zeros(len(t))
    X_err[0]=sigma
    for i in range(1,len(t)):
        dt[i]=t[i]-t[i-1]
        E_mag[i]=np.exp(-dt[i]/tau)*X[i-1]+mu*(1-np.exp(-dt[i]/tau))
        var[i]=var[0]*(1-np.exp(-2*dt[i]/tau)) 
        X[i]=E_mag[i]+np.random.standard_normal()*np.sqrt(var[i])
        
    return X

print 'start'
time=range(365)

n=range(100)
tau_obs=np.zeros(len(n))
sig_obs=np.zeros_like(tau_obs)
tau_obs2=np.zeros_like(tau_obs)
sig_obs2=np.zeros_like(tau_obs)
tau_zu=np.zeros_like(tau_obs)
sig_zu=np.zeros_like(tau_obs)


time_real=np.genfromtxt('/Users/legolason/study/lightcurve/SF_test/data/1310_copy')
time_real=time_real-time_real.min()
idx=time_real.astype(int)


for i in n:
    X=model_lc(time,17,100,0.1)
    
    '''
    plt.figure(0)
    plt.plot(np.array(time),X)
    plt.plot(idx,X[idx],'ro')
    #plt.show()
    '''
    model = cm.CarmaModel(np.array(time), X,np.random.rand(len(X))/100., p=1, q=0)
    sample = model.run_mcmc(5000)
    tau_obs[i]=np.exp(-np.median(sample.get_samples('log_omega')))
    sig_obs[i]=np.median(sample.get_samples('sigma'))
    print tau_obs[i],sig_obs[i]
    '''
    model2 = cm.CarmaModel(idx, X[idx],np.random.rand(len(X[idx]))/100., p=1, q=0)
    sample2 = model2.run_mcmc(10000)
    tau_obs2[i]=((np.exp(-np.median(sample2.get_samples('log_omega')))))
    SF_obs2[i]=np.median(sample2.get_samples('sigma'))*np.sqrt(tau_obs2[i])
    print tau_obs2[i],SF_obs2[i]
    
    plt.figure(2,figsize=(12,8))
    plt.subplot(121)
    plt.hist(np.log10(sample2.get_samples('sigma')*np.sqrt(tau_obs2[i]/2.)),100)
    plt.xlabel(r'$log {\sigma}$')
    plt.ylabel('N')
    plt.subplot(122)
    plt.hist(np.log10(np.exp(-sample2.get_samples('log_omega'))),100)
    plt.xlabel(r'$log{\tau}$')
    plt.ylabel('N')
    plt.figure(3)
    sample2.assess_fit()
    plt.show()
    '''
    
    np.savetxt('tmp1.txt',np.c_[np.array(time), X,np.random.rand(len(X))/100.])
    javdata1 = get_data('tmp1.txt')
    cont1 = Cont_Model(javdata1)
    cont1.do_mcmc(fchain="mychain1.dat")
    cont1.hpd
    tau_zu[i]=np.exp(cont1.hpd[1,1])
    sig_zu[i]=np.exp(cont1.hpd[1,0])

   
    
    '''
    np.savetxt('tmp.txt',np.c_[idx, X[idx],np.random.rand(len(X[idx]))/100.])
    javdata = get_data('tmp.txt')
    cont = Cont_Model(javdata)
    cont.do_mcmc(fchain="mychain0.dat")
    cont.show_hist(bins=100)
    javdata_best =  cont.do_pred(cont.hpd[1,:])
    javdata_best.plot(set_pred=True, obs=javdata)
    '''
    print i
    
np.savetxt('/Users/legolason/study/lightcurve/SF_test/comparison/lc_whole/lc_whole.txt',\
           np.c_[tau_obs,sig_obs,tau_zu,sig_zu])
    

plt.figure(figsize=(10,6))
plt.subplot(121)
plt.hist(np.log10(tau_obs),histtype='step',label='kel09')
plt.hist(np.log10(tau_zu),histtype='step',label='zu11')
plt.ylabel('N')
plt.xlabel(r'$\rm log \tau$ ')
plt.legend()
plt.subplot(122)
plt.hist(np.log10(sig_obs),histtype='step',label='kel09')
plt.hist(np.log10(sig_zu*1.414/np.sqrt(tau_zu)),histtype='step',label='zu11')
plt.ylabel('N')
plt.xlabel(r'$\rm log \sigma$ ')
plt.legend()
plt.savefig('/Users/legolason/study/lightcurve/SF_test/comparison/lc_whole/lc_whole.eps')
print np.median(tau_obs),np.median(sig_obs)
print np.median(tau_zu),np.median(sig_zu)


In [ ]:
# test lightcurve compare two methods!  test 2nd!
%matplotlib inline
import carmcmc as cm
import numpy as np
import matplotlib.pyplot as plt
import os,glob
from javelin.zylc import get_data
from javelin.lcmodel import Cont_Model

def model_lc(time, mu, tau, sigma):
    '''re-generate a new light cuvre with given tau, sigma and mu '''
    t=time
    E_mag=np.zeros(len(t))
    E_mag[0]=mu
    var=np.zeros(len(t))
    var[0]=tau*sigma**2/2.
    X=np.zeros(len(t))
    X[0]=mu
    dt=np.zeros(len(t))
    X_err=np.zeros(len(t))
    X_err[0]=sigma
    for i in range(1,len(t)):
        dt[i]=t[i]-t[i-1]
        E_mag[i]=np.exp(-dt[i]/tau)*X[i-1]+mu*(1-np.exp(-dt[i]/tau))
        var[i]=var[0]*(1-np.exp(-2*dt[i]/tau)) 
        X[i]=E_mag[i]+np.random.standard_normal()*np.sqrt(var[i])
        
    return X

print 'start'
plt.rc('font', size=8)
time=range(3650)

n=range(100)
tau_obs=np.zeros(len(n))
sig_obs=np.zeros_like(tau_obs)
tau_obs2=np.zeros_like(tau_obs)
sig_obs2=np.zeros_like(tau_obs)
tau_zu=np.zeros_like(tau_obs)
sig_zu=np.zeros_like(tau_obs)


time_real=np.genfromtxt('/Users/legolason/study/lightcurve/SF_test/data/1310_copy')
time_real=time_real-time_real.min()
idx=time_real.astype(int)


for i in n:
    X=model_lc(time,17,100,0.1)
    
    
    plt.figure(0)
    plt.plot(np.array(time),X)
    plt.plot(idx,X[idx],'ro')
    #plt.show()
    
    model = cm.CarmaModel(np.array(time), X,np.random.rand(len(X))/100., p=1, q=0)
    sample = model.run_mcmc(5000)
    tau_obs[i]=np.exp(-np.median(sample.get_samples('log_omega')))
    sig_obs[i]=np.median(sample.get_samples('sigma'))
    print tau_obs[i],sig_obs[i]
    
    model2 = cm.CarmaModel(idx, X[idx],np.random.rand(len(X[idx]))/100., p=1, q=0)
    sample2 = model2.run_mcmc(10000)
    tau_obs2[i]=((np.exp(-np.median(sample2.get_samples('log_omega')))))
    sig_obs2[i]=np.median(sample2.get_samples('sigma'))
    print tau_obs2[i],sig_obs2[i]
    
    plt.figure(2,figsize=(12,8))
    plt.subplot(121)
    plt.hist(np.log10(sample2.get_samples('sigma')),100)
    plt.xlabel(r'$log {\sigma}$')
    plt.ylabel('N')
    plt.subplot(122)
    plt.hist(np.log10(np.exp(-sample2.get_samples('log_omega'))),100)
    plt.xlabel(r'$log{\tau}$')
    plt.ylabel('N')
    plt.figure(3)
    sample2.assess_fit()
    plt.show()
    
    '''
    np.savetxt('tmp1.txt',np.c_[np.array(time), X,np.random.rand(len(X))/100.])
    javdata1 = get_data('tmp1.txt')
    cont1 = Cont_Model(javdata1)
    cont1.do_mcmc(fchain="mychain1.dat")
    cont1.hpd
    tau_zu[i]=np.exp(cont1.hpd[1,1])
    sig_zu[i]=np.exp(cont1.hpd[1,0])
    '''
   
    
    '''
    np.savetxt('tmp.txt',np.c_[idx, X[idx],np.random.rand(len(X[idx]))/100.])
    javdata = get_data('tmp.txt')
    cont = Cont_Model(javdata)
    cont.do_mcmc(fchain="mychain0.dat")
    cont.show_hist(bins=100)
    javdata_best =  cont.do_pred(cont.hpd[1,:])
    javdata_best.plot(set_pred=True, obs=javdata)
    '''
    print i
    
np.savetxt('/Users/legolason/study/lightcurve/SF_test/comparison/kel09_samp_wh/samp_wh.txt',\
           np.c_[tau_obs,sig_obs,tau_obs2,sig_obs2],header='tau_obs,sig_obs,tau_obs2,sig_obs2')
    

plt.subplot(121)
plt.hist(np.log10(tau_obs),histtype='step',label='whole')
plt.hist(np.log10(tau_obs2),histtype='step',label='sampling')
plt.ylabel('N')
plt.xlabel(r'$\rm log \tau$ ')

plt.subplot(122)
plt.hist(np.log10(sig_obs),histtype='step',label='whole')
plt.hist(np.log10(sig_obs2),histtype='step',label='sampling')
plt.ylabel('N')
plt.xlabel(r'$\rm log \sigma$ ')

plt.savefig('/Users/legolason/study/lightcurve/SF_test/comparison/kel09_samp_wh/samp_wh.eps')
print np.median(tau_obs),np.median(sig_obs)
print np.median(tau_obs2),np.median(sig_obs2)


In [ ]:
# test lightcurve compare two methods!  test 3rd!
%matplotlib inline
import carmcmc as cm
import numpy as np
import matplotlib.pyplot as plt
import os,glob
from javelin.zylc import get_data
from javelin.lcmodel import Cont_Model

def model_lc(time, mu, tau, sigma):
    '''re-generate a new light cuvre with given tau, sigma and mu '''
    t=time
    E_mag=np.zeros(len(t))
    E_mag[0]=mu
    var=np.zeros(len(t))
    var[0]=tau*sigma**2/2.
    X=np.zeros(len(t))
    X[0]=mu
    dt=np.zeros(len(t))
    X_err=np.zeros(len(t))
    X_err[0]=sigma
    for i in range(1,len(t)):
        dt[i]=t[i]-t[i-1]
        E_mag[i]=np.exp(-dt[i]/tau)*X[i-1]+mu*(1-np.exp(-dt[i]/tau))
        var[i]=var[0]*(1-np.exp(-2*dt[i]/tau)) 
        X[i]=E_mag[i]+np.random.standard_normal()*np.sqrt(var[i])
        
    return X

print 'start'
plt.rc('font', size=8)
time=range(365)

n=range(100)
tau_obs=np.zeros(len(n))
sig_obs=np.zeros_like(tau_obs)
tau_obs2=np.zeros_like(tau_obs)
sig_obs2=np.zeros_like(tau_obs)
tau_zu=np.zeros_like(tau_obs)
sig_zu=np.zeros_like(tau_obs)
tau_zu2=np.zeros_like(tau_obs)
sig_zu2=np.zeros_like(tau_obs)

time_real=np.genfromtxt('/Users/legolason/study/lightcurve/SF_test/data/1310_copy')
time_real=time_real-time_real.min()
idx=time_real.astype(int)/10


for i in n:
    X=model_lc(time,17,100,0.1)
    
    '''
    plt.figure(0)
    plt.plot(np.array(time),X)
    plt.plot(idx,X[idx],'ro')
    #plt.show()
    
    model = cm.CarmaModel(np.array(time), X,np.random.rand(len(X))/100., p=1, q=0)
    sample = model.run_mcmc(5000)
    tau_obs[i]=np.exp(-np.median(sample.get_samples('log_omega')))
    sig_obs[i]=np.median(sample.get_samples('sigma'))
    print tau_obs[i],sig_obs[i]
    
    model2 = cm.CarmaModel(idx, X[idx],np.random.rand(len(X[idx]))/100., p=1, q=0)
    sample2 = model2.run_mcmc(10000)
    tau_obs2[i]=((np.exp(-np.median(sample2.get_samples('log_omega')))))
    sig_obs2[i]=np.median(sample2.get_samples('sigma'))
    print tau_obs2[i],sig_obs2[i]
    
    plt.figure(2,figsize=(12,8))
    plt.subplot(121)
    plt.hist(np.log10(sample2.get_samples('sigma')),100)
    plt.xlabel(r'$log {\sigma}$')
    plt.ylabel('N')
    plt.subplot(122)
    plt.hist(np.log10(np.exp(-sample2.get_samples('log_omega'))),100)
    plt.xlabel(r'$log{\tau}$')
    plt.ylabel('N')
    plt.figure(3)
    sample2.assess_fit()
    plt.show()
    
    '''
    np.savetxt('tmp1.txt',np.c_[np.array(time), X,np.random.rand(len(X))/100.])
    javdata1 = get_data('tmp1.txt')
    cont1 = Cont_Model(javdata1)
    cont1.do_mcmc(fchain="mychain1.dat",set_prior=False)
    tau_zu[i]=np.exp(cont1.hpd[1,1])
    sig_zu[i]=np.exp(cont1.hpd[1,0])
   
   
    
    
    np.savetxt('tmp2.txt',np.c_[idx, X[idx],np.random.rand(len(X[idx]))/100.])
    javdata = get_data('tmp2.txt')
    cont2 = Cont_Model(javdata)
    cont2.do_mcmc(fchain="mychain2.dat",set_prior=False)
    cont2.show_hist(bins=100)
    javdata_best =  cont2.do_pred(cont2.hpd[1,:])
    javdata_best.plot(set_pred=True, obs=javdata)
    tau_zu2[i]=np.exp(cont2.hpd[1,1])
    sig_zu2[i]=np.exp(cont2.hpd[1,0])
    
    print i
    
np.savetxt('/Users/legolason/study/lightcurve/SF_test/comparison/zu11_samp_wh/samp_wh.txt',\
           np.c_[tau_zu,sig_zu2,tau_zu,sig_zu2])
    

plt.subplot(121)
plt.hist(np.log10(tau_zu),histtype='step',label='zu_whole')
plt.hist(np.log10(tau_zu2),histtype='step',label='zu_sampling')
plt.ylabel('N')
plt.xlabel(r'$\rm log \tau$ ')
plt.legend(prop={'size':8})


plt.subplot(122)
plt.hist(np.log10(sig_zu*1.414/np.sqrt(tau_zu)),histtype='step',label='zu_whole')
plt.hist(np.log10(sig_zu2*1.414/np.sqrt(tau_zu2)),histtype='step',label='zu_sampling')
plt.ylabel('N')
plt.xlabel(r'$\rm log \sigma$ ')
plt.legend(prop={'size':8})

plt.savefig('/Users/legolason/study/lightcurve/SF_test/comparison/zu11_samp_wh/samp_wh.eps')
#print np.median(tau_obs),np.median(sig_obs)
#print np.median(tau_obs2),np.median(sig_obs2)


In [ ]:
# test lightcurve compare two methods!  test 4th!
%matplotlib inline
import carmcmc as cm
import numpy as np
import matplotlib.pyplot as plt
import os,glob
from javelin.zylc import get_data
from javelin.lcmodel import Cont_Model

def model_lc(time, mu, tau, sigma):
    '''re-generate a new light cuvre with given tau, sigma and mu '''
    t=time
    E_mag=np.zeros(len(t))
    E_mag[0]=mu
    var=np.zeros(len(t))
    var[0]=tau*sigma**2/2.
    X=np.zeros(len(t))
    X[0]=mu
    dt=np.zeros(len(t))
    X_err=np.zeros(len(t))
    X_err[0]=sigma
    for i in range(1,len(t)):
        dt[i]=t[i]-t[i-1]
        E_mag[i]=np.exp(-dt[i]/tau)*X[i-1]+mu*(1-np.exp(-dt[i]/tau))
        var[i]=var[0]*(1-np.exp(-2*dt[i]/tau)) 
        X[i]=E_mag[i]+np.random.standard_normal()*np.sqrt(var[i])
        
    return X

print 'start'
plt.rc('font', size=8)
time=range(3650)

n=range(1000)
tau_obs=np.zeros(len(n))
sig_obs=np.zeros_like(tau_obs)
tau_obs2=np.zeros_like(tau_obs)
sig_obs2=np.zeros_like(tau_obs)
tau_zu=np.zeros_like(tau_obs)
sig_zu=np.zeros_like(tau_obs)
tau_zu2=np.zeros_like(tau_obs)
sig_zu2=np.zeros_like(tau_obs)

time_real=np.genfromtxt('/Users/legolason/study/lightcurve/SF_test/data/233_copy')
time_real=time_real-time_real.min()
idx=time_real.astype(int)


for i in n:
    X=model_lc(time,17,100,0.1)
    
    '''
    plt.figure(0)
    plt.plot(np.array(time),X)
    plt.plot(idx,X[idx],'ro')
    #plt.show()
    
    model = cm.CarmaModel(np.array(time), X,np.random.rand(len(X))/100., p=1, q=0)
    sample = model.run_mcmc(5000)
    tau_obs[i]=np.exp(-np.median(sample.get_samples('log_omega')))
    sig_obs[i]=np.median(sample.get_samples('sigma'))
    print tau_obs[i],sig_obs[i]
    '''
    model2 = cm.CarmaModel(idx, X[idx],np.random.rand(len(X[idx]))/100., p=1, q=0)
    sample2 = model2.run_mcmc(5000)
    tau_obs2[i]=((np.exp(-np.median(sample2.get_samples('log_omega')))))
    sig_obs2[i]=np.median(sample2.get_samples('sigma'))
    print tau_obs2[i],sig_obs2[i]
    
    plt.figure(2,figsize=(12,8))
    plt.subplot(121)
    plt.hist(np.log10(sample2.get_samples('sigma')),100)
    plt.xlabel(r'$log {\sigma}$')
    plt.ylabel('N')
    plt.subplot(122)
    plt.hist(np.log10(np.exp(-sample2.get_samples('log_omega'))),100)
    plt.xlabel(r'$log{\tau}$')
    plt.ylabel('N')
    plt.figure(3)
    sample2.assess_fit()
    plt.show()
    
    '''
    np.savetxt('tmp1.txt',np.c_[np.array(time), X,np.random.rand(len(X))/100.])
    javdata1 = get_data('tmp1.txt')
    cont1 = Cont_Model(javdata1)
    cont1.do_mcmc(fchain="mychain1.dat")
    tau_zu[i]=np.exp(cont1.hpd[1,1])
    sig_zu[i]=np.exp(cont1.hpd[1,0])
    '''
   
    
    
    np.savetxt('tmp2.txt',np.c_[idx, X[idx],np.random.rand(len(X[idx]))/100.])
    javdata = get_data('tmp2.txt')
    cont2 = Cont_Model(javdata)
    cont2.do_mcmc(fchain="mychain2.dat")
    cont2.show_hist(bins=100)
    javdata_best =  cont2.do_pred(cont2.hpd[1,:])
    javdata_best.plot(set_pred=True, obs=javdata)
    tau_zu2[i]=np.exp(cont2.hpd[1,1])
    sig_zu2[i]=np.exp(cont2.hpd[1,0])
    
    print i
    
np.savetxt('/Users/legolason/study/lightcurve/SF_test/comparison/lc_samp/lc_samp.txt',\
           np.c_[tau_obs2,sig_zu2,tau_obs2,sig_zu2],header='tau_obs2,sig_zu2,tau_obs2,sig_zu2 (obs frame)')
    

plt.subplot(121)
plt.hist(np.log10(tau_obs2),histtype='step',label='kel09')
plt.hist(np.log10(tau_zu2),histtype='step',label='zu11')
plt.ylabel('N')
plt.xlabel(r'$\rm log \tau$ ')
plt.legend(prop={'size':8})


plt.subplot(122)
plt.hist(np.log10(sig_obs2),histtype='step',label='kel09')
plt.hist(np.log10(sig_zu2*np.sqrt(2)/np.sqrt(tau_zu2)),histtype='step',label='zu11')
plt.ylabel('N')
plt.xlabel(r'$\rm log \sigma$ ')
plt.legend(prop={'size':8})

plt.savefig('/Users/legolason/study/lightcurve/SF_test/comparison/lc_samp/lc_samp_1000.eps')
#print np.median(tau_obs),np.median(sig_obs)
#print np.median(tau_obs2),np.median(sig_obs2)
